In [1]:
#spark.stop()

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpt
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from graphframes import *
from delta import *
from delta.tables import *

import pyspark.sql.functions as fn
from pyspark.sql.types import StringType, BooleanType, DateType

Define the configuration that should be used

In [3]:
config = pyspark.SparkConf().setAll([
    ('spark.executor.memory', '12g'), 
    ('spark.executor.cores', '3'), 
    ('spark.cores.max', '6'),
    ('spark.driver.memory','1g'),
    ('spark.executor.instances', '1'),
    ('spark.dynamicAllocation.enabled', 'true'),
    ('spark.dynamicAllocation.shuffleTracking.enabled', 'true'),
    ('spark.dynamicAllocation.executorIdleTimeout', '60s'),
    ('spark.dynamicAllocation.minExecutors', '0'),
    ('spark.dynamicAllocation.maxExecutors', '2'),
    ('spark.dynamicAllocation.initialExecutors', '1'),
    ('spark.dynamicAllocation.executorAllocationRatio', '1')
])

In [4]:
builder = pyspark.sql.SparkSession.builder \
    .appName("UseCaseBased Transactions") \
    .master("spark://172.23.149.212:7077") \
    .config(conf=config) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

22/06/27 16:31:14 WARN Utils: Your hostname, algorand-druid-and-spark resolves to a loopback address: 127.0.0.1; using 172.23.149.212 instead (on interface ens3)
22/06/27 16:31:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-580294cd-181a-4c93-bc55-746e97f64f2c;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 291ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      d

Read the transactions_flat table from the Delta Lake

In [5]:
# account table to determine which accounts have received rewards
dfTx = spark.read.format("delta").load("/mnt/delta/bronze/algod_indexer_public_txn_flat")
dfTx = dfTx.drop("key")
dfTx = dfTx.repartition(6)
dfTx.rdd.getNumPartitions()

print("Number of total transactions", dfTx.count())
dfTx.show(1, vertical=True, truncate=False)

AnalysisException: `/mnt/delta/bronze/algod_indexer_public_txn_flat` is not a Delta table.

### Use Cases

We want to distinguish between different use cases of the transactions. To do so, we make use of type of a transaction. we distinguish between the following transactions
- PaymentTx, type = "pay", typeEnum = 1. This transaction sends Algos from one account to another.
- KeyRegistrationTx, type = "keyreg", typeEnum = 2. This transaction is done to register an account either online or offline. A transaction is an online transaction if it has participation-key related fields, namely votekey, selkey, votekd, votefst and votelst. A transaction is an offline transaction if these fields are missing. The moment a key registration transaction is confirmed by the network it takes 320 rounds for the change to take effect. In other words, if a key registration is confirmed in round 1000, the account will not start participating until round 1320.
- AssetConfigTx, type = "acfg", typeEnum = 3. This transaction is used to create an asset, modify certain parameters of an asset, or destroy an asset. 
- AssetTransferTx, type = "axfer", typeEnum = 4. An Asset Transfer Transaction is used to opt-in to receive a specific type of Algorand Standard Asset, transfer an Algorand Standard asset, or revoke an Algorand Standard Asset from a specific account.
- AssetFreezeTx, type = "afrz", tpyeEnum = 5. An Asset Freeze Transaction is issued by the Freeze Address and results in the asset receiver address losing or being granted the ability to send or receive the frozen asset.
- ApplicationCallTx, type = "appl", typeEnum = 6. An Application Call Transaction is submitted to the network with an AppId and an OnComplete method. The AppId specifies which App to call and the OnComplete method is used in the contract to determine what branch of logic to execute.

More information can be found under https://developer.algorand.org/docs/get-details/transactions/

In [ ]:
dfUseCaseDistribution = dfTx.groupBy("TXN_TYPE").count()
dfUseCaseDistribution.show()

Create a cross join to have another column with the sum in each row for diving it later

In [ ]:
dfUseCaseDistribution = dfUseCaseDistribution.crossJoin(dfUseCaseDistribution.groupby().agg(fn.sum('count').alias('sum_count')))

In [ ]:
dfUseCaseDistribution = dfUseCaseDistribution.select('TXN_TYPE', 'count', (fn.col('count') / fn.col('sum_count')).alias("percent"))

In [ ]:
dfUseCaseDistribution.show(truncate=False)

In [ ]:
dfMiners.write.format("delta").mode("overwrite").save("/mnt/delta/silver/queries/network/miner.addresses")
dfUsers.write.format("delta").mode("overwrite").save("/mnt/delta/silver/queries/network/user.addresses")

22/06/27 16:31:43 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
22/06/27 16:31:43 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:919)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:154)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:262)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:169)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:100)
	at org.apache.spark.rpc.netty.MessageLoop.org$apache$spark$rpc$netty$Mess